In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
from sklearn import metrics
from keras.layers import Activation
from keras.models import model_from_json
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
import numpy
import pandas
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:
numpy.random.seed(1)

print('loading TRAIN data')

dataframe_TRAIN = pandas.read_csv("train_newcsd_neutron.txt", delimiter=",", header=None)
dataset_TRAIN=dataframe_TRAIN.values
# split into input (X) and output (Y) variables

X_TRAIN = dataset_TRAIN[:,0:33]
Y_TRAIN = dataset_TRAIN[:,33]


dataframe_test = pandas.read_csv("test_newcsd_neutron.txt", delimiter=",", header=None)
dataset_test=dataframe_test.values

X_test = dataset_test[:,0:33]
Y_test = dataset_test[:,33]


In [ ]:
seed = 777
#test_size = 0.25
#test_size = 0.4
test_size = 0.5

X_TRAIN, Y_TRAIN = shuffle( X_TRAIN, Y_TRAIN, random_state = seed )
X_train, X_val, Y_train, Y_val= train_test_split(X_TRAIN, Y_TRAIN, test_size=test_size, random_state=seed)

In [ ]:
print('building network')

#l2val=0.01
#neurons=256
#drop=0.1

l2val=0.003
neurons=256
drop=0.5

class_weight = {0: 1.0,
                1: 1.0,
               }

activation='relu'
#activation='tanh'

initial='glorot_uniform'
model = Sequential()
#model.add(Dense(neurons, input_dim=31,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dense(neurons, input_dim=33,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#1
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#2
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#3
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#4
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#5
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#6
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#7
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#8
model.add(Dense(neurons,kernel_initializer=initial,kernel_regularizer=regularizers.l2(l2val)))
model.add(Dropout(drop))
model.add(Activation(activation))

#
model.add(Dense(1,kernel_initializer=initial))
model.add(Activation('sigmoid'))

#adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0001)

epochs = 300
learning_rate = 0.001
decay_rate = learning_rate / epochs * 10
adam=optimizers.Adam(lr=learning_rate, decay=decay_rate, beta_1=0.9, beta_2=0.999, epsilon=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

history=model.fit(X_train, Y_train,validation_data=(X_val,Y_val), epochs=epochs, batch_size=2048,shuffle=False,class_weight=class_weight)



In [ ]:
model_json = model.to_json()
with open("model_eta_train19.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_eta_train19.h5")
print("Saved model to disk")

In [ ]:
print('plotting history')

print(history.history.keys())

fig = plt.figure()
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
fig.savefig("history.pdf")

In [ ]:
scores = model.evaluate(X_val, Y_val)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

probs_train = model.predict_proba(X_train)
preds_train = probs_train[:,0]
fpr_train, tpr_train, threshold_train = metrics.roc_curve(Y_train, preds_train)

probs_val = model.predict_proba(X_val)
preds_val = probs_val[:,0]
fpr_val, tpr_val, threshold_val = metrics.roc_curve(Y_val, preds_val)

probs_test = model.predict_proba(X_test)
preds_test = probs_test[:,0]
fpr_test, tpr_test, threshold_test = metrics.roc_curve(Y_test, preds_test)

# probs_TRAIN=model.predict_proba(X_TRAIN)
# preds_TRAIN = probs_TRAIN[:,0]
# fpr_TRAIN, tpr_TRAIN, threshold_TRAIN = metrics.roc_curve(Y_TRAIN, preds_TRAIN)

In [ ]:
print('plotting training results')
fig = plt.figure()
plt.title('Gamma-Level ROC for EtaCSD cut')
plt.plot(tpr_val, 1-fpr_val, 'b',label='val')
# plt.plot(tpr_test, 1-fpr_test, 'r',label='test')
plt.plot(tpr_train, 1-fpr_train, 'green',label='train')
plt.legend(loc = 'lower left')
#plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('signal efficiency')
plt.ylabel('background rejection')
plt.show()
fig.savefig("ROC.pdf")

In [ ]:
fig = plt.figure()
plt.title('Gamma-Level ROC for EtaCSD cut')
plt.plot(tpr_val, 1-fpr_val, 'b',label='val')
plt.plot(tpr_test, 1-fpr_test, 'r',label='test')
plt.plot(tpr_train, 1-fpr_train, 'green',label='train')
plt.legend(loc = 'lower left')
plt.xlim([0.895, 0.905])
plt.ylim([0.99, 0.998])
plt.xlabel('signal efficiency')
plt.ylabel('background rejection')
plt.show()
fig.savefig("ROC_zoom.pdf")

In [ ]:
fig = plt.figure()
plt.title('Gamma-Level ROC for EtaCSD cut')
plt.plot(tpr_test,threshold_test, 'r',label='test')
plt.legend(loc = 'lower left')
plt.xlim([0.89, 0.91])
plt.ylim([0.8, 0.86])
plt.ylabel('threshold')
plt.xlabel('signal efficiency')
plt.show()
fig.savefig("ROC_zoom.pdf")

In [ ]:
print('loading Original CSD data')

dataframe_test_csd=pandas.read_csv("csd1.txt", delimiter=",", header=None)
dataset_test_csd=dataframe_test_csd.values
preds_test_csd= dataset_test_csd[:,0]
Y_test_csd= dataset_test_csd[:,1]

fpr_test_csd, tpr_test_csd, threshold_test_csd = metrics.roc_curve(Y_test_csd, preds_test_csd)

In [ ]:
print('plotting comparison with csd0 on TRAIN data set')

fig = plt.figure()
plt.title('comparison on test data group')
# plt.plot(tpr_TRAIN, 1-fpr_TRAIN, 'r',label='EtaCSD_TRAIN')
# plt.plot(tpr_val, 1-fpr_val, 'r',label='NewCSD_TRAIN')
plt.plot(tpr_test, 1-fpr_test, 'r',label='GPU CSD')
plt.plot(tpr_test_csd, 1-fpr_test_csd, 'black',label='ROOT CSD')
# plt.plot(tpr_test_csd, 1-fpr_test_csd, 'violet',label='MinCSD_test')
plt.legend(loc = 'lower left')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('signal efficiency')
plt.ylabel('background rejection')
plt.show()
fig.savefig("comparison_on_test.pdf")

fig = plt.figure()
plt.title('comparison on test data group zoom')
# plt.plot(tpr_test, 1-fpr_test, 'r',label='NewCSD_test')
# plt.plot(tpr_val, 1-fpr_val, 'r',label='NewCSD_test')
plt.plot(tpr_test, 1-fpr_test, 'r',label='GPU CSD')
plt.plot(tpr_test_csd, 1-fpr_test_csd, 'black',label='ROOT CSD')
# plt.plot(tpr_test_csd, 1-fpr_test_csd, 'violet',label='MinCSD_test')
plt.legend(loc = 'lower left')
plt.xlim([0.89, 0.91])
plt.ylim([0.98, 1.0])
plt.xlabel('signal efficiency')
plt.ylabel('background rejection')
plt.show()
fig.savefig("comparison_on_test_zoom.pdf")